#Estudiante: Roy Chavarría Garita
##Semana #1

###Ejercicios
-Replicar lo visto en el video https://www.youtube.com/watch?v=eeLch59fU78

In [1]:
!pip install torch numpy

In [2]:
###Bibliotecas
import torch
import numpy as np

Un modelo es lo que trata de imitar un ser humano dentro de una red neuronal. Ejemplo: predecir toneladas de manzanas y naranjas producidas en una región. ESto se hace en base a parámetros como lluvia, humedad, temeperatura, etc. \\
A cada uno de esos parametros se multiplican por unos pesos y se les suma una constante. Ejemplo: \\
Manzanas = w1*temperatura+w2*lluvia+w3*humedad + b1 \\
Con datos previos, el modelo podrá predecir correctamente.


In [3]:
 #Temperatura, lluvia, humedad
 entradas = np.array([[73,67,43],
                     [91,88,64],
                     [87,134,58],
                     [102,43,37],
                     [69,96,70]], dtype='float32')

#(Manzanas o naranjas)
resultados = np.array([[56,70],
                       [81,101],
                       [119,133],
                       [22,37],
                       [103,119]],dtype='float32')


#Los datos que leo normalmente es en un csv, pero como este es un ejemplo
#vendrán en esas matrices, ojo que son muy pocos datos

In [4]:
#Convierto entradas y salidas a tensores
entradas = torch.from_numpy(entradas)
resultados = torch.from_numpy(resultados)
print(entradas)
print(resultados)


tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


##Creación del modelo

Recordando por ejemplo los pesos y sesgos para la manzana: Manzanas = w1temperatura+w2lluvia+w3*humedad + b1

In [5]:
pesos = torch.rand(2,3,requires_grad=True)
sesgos = torch.rand(2, requires_grad=True)

print(pesos)
print(sesgos)


tensor([[0.3993, 0.6331, 0.5668],
        [0.8458, 0.0179, 0.7895]], requires_grad=True)
tensor([0.6140, 0.5374], requires_grad=True)


In [6]:
#Genero el modelo
def model(entrada):
  return entrada @ pesos.t() + sesgos

In [7]:
predicciones = model(entradas)
print(predicciones)
print(resultados)

#Hay una prediccion bastante mala, debemos ir ajustando pesos y sesgos

tensor([[ 96.5500,  97.4341],
        [128.9340, 129.6158],
        [153.0571, 122.3198],
        [ 89.5359, 116.7957],
        [128.6143, 115.8881]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Como lo anterior me predice bastante mal, voy a utilizar el algoritmo MSE (error cuadratico medio), el cual me ayuda a ir ajustando los pesos y sesgos para mejorar mi predicción.
Este algoritmo tiene 3 pasos: \\
- Calcular diferencia entre matriz de predicciones y la de resultados.
- Elevar al cuadrado la matriz resultante para eliminar negativos.
-Calcular el valor promedio de los elementos de la matriz.

In [8]:
def mse(m1,m2):
  diff = m1 - m2
  return torch.sum(diff * diff) / diff.numel() #numel es para que me tire resultado en tensores

In [9]:
loss = mse(predicciones, resultados)
print(loss)


tensor(1838.1647, grad_fn=<DivBackward0>)


Loss en este momento es muy tonta, no predice bien por lo que debemos ajustar pesos y sesgos para que ese loss se aproxime a 0. Esto lo vamos a hacer con el calculo de gradiente.
Calculamos gradiente respecto a peso y eso nos ayudará a reducir loss. \\
-Si gradiente de un peso es positivo, un incremento en ese peso aumenta el loss y un decremento lo disminuye.
- Si el gradiente de un peso es negativo, un incremento en el valor del peso disminuye loss y una reduccion aumenta el loss.

In [10]:
#Guardo las gradientes en cada uno de los pesos
loss.backward()

Ya con los pesos guardados, tengo que ir ajustandolos poco a poco, no puedo dar pasos grandes. Una opción que podemos hacer es sustraer a cada peso una cantidad pequeña proporcional a la derivada de ese peso.

In [11]:
with torch.no_grad():
  pesos -= pesos.grad * 0.00005
  sesgos -= sesgos.grad * 0.00005


In [12]:
predicciones = model(entradas)
loss = mse(predicciones,resultados)
print(loss)


tensor(752.4502, grad_fn=<DivBackward0>)


In [13]:
pesos.grad.zero_()
sesgos.grad.zero_()
print(pesos.grad)
print(sesgos.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


##Resumen
1. Generar las predicciones
2. Calcular el loss
3. Calcular las gradientes con respecto a pesos y sesgos
4. Ajustar los pesos sustrayeno de ellos una pequeña cantidad proporcional al gradiente
5. Reiniciar gradientes a 0

Todo esto hay que hacerlo varias veces, esto se llama ciclo de entrenamiento, con varios ciclos el loss va disminuyendo